In [1]:
from img_doc.dataset import PubLayNetDataset
from img_doc.document import Document

2024-02-28 11:18:04.865749: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-28 11:18:04.892753: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 11:18:04.892781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 11:18:04.893528: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-28 11:18:04.897947: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-28 11:18:04.898372: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
# Практически равны
# pln_ds = PubLayNetDataset(r"/home/daniil/program/dataset/publaynet/")
# pln_ds.create_tmp_annotation_jsons(r"/home/daniil/program/dataset/tmp_pln2", lambda x: {}, 0, 2)
# pln_ds = PubLayNetDataset(r"/home/daniil/program/dataset/publaynet/")
# pln_ds.create_tmp_annotation_jsons(r"/home/daniil/program/dataset/tmp_pln3", lambda x: {}, 0, 1)
# pln_ds.create_tmp_annotation_jsons(r"/home/daniil/program/dataset/tmp_pln3", lambda x: {}, 1, 2)

In [62]:
def fun_additional_info(img_path):
    doc = Document()
    doc.set_from_path(img_path)
    doc.pages[0].extract_word(conf={"lang": "eng", "psm": 4, "oem": 3, "k": 4})
    json_words = [w.to_dict()  for w in doc.pages[0].words]
    return {"words": json_words}

pln_ds = PubLayNetDataset(r"/home/daniil/program/dataset/publaynet/")
pln_ds.create_tmp_annotation_jsons(r"/home/daniil/program/dataset/tmp_publaynet", fun_additional_info, 2500, 5000)
#пропустил 1300-1500
#...-2500

кол-во изображений:	 10658
кол-во сегментов:	 10658


In [63]:
from img_doc.document import Document, Block
from img_doc.document.data_structures.page.extractors.block_extractors import BaseRandomWalkClassificator

import numpy as np
import json
import random

In [64]:
pln_ds = PubLayNetDataset(r"/home/daniil/program/dataset/publaynet/", 
                          r"/home/daniil/program/dataset/tmp_publaynet/")

In [65]:
rnd_cls = BaseRandomWalkClassificator(properties=["many_dist", "many_angle", "place_in_block"],count_step=50)


In [66]:
def fun_from_tmp_and_path_image(tmp_json, img_path):
    doc = Document()
    doc.set_from_path(img_path)
    doc.pages[0].set_blocks_from_dict(tmp_json["blocks"])
    doc.pages[0].set_words_from_dict(tmp_json["additional_info"]["words"])
    for word in doc.pages[0].words:
        for block in doc.pages[0].blocks:
            if block.segment.is_intersection(word.segment):
                block.words.append(word)
    x = []
    y = []
    for page in doc.pages:
        for block in page.blocks:
            try:
                x.append(rnd_cls.get_block_vec(block))
                y.append(block.label)
            except:
                t = block.get_text()
                if len(t) > 10:
                    text_ = t[:10] + "..."
                else:
                    text_ = t
                print(doc.path, text_)
    return {"x": x, "y": y}


rez = pln_ds.get_json_from_tmps_and_images(fun_from_tmp_and_path_image)

train:
/home/daniil/program/dataset/publaynet/train/PMC3505796_00004.jpg 
/home/daniil/program/dataset/publaynet/train/PMC5919984_00000.jpg Keywords F...
/home/daniil/program/dataset/publaynet/train/PMC5800348_00000.jpg 
/home/daniil/program/dataset/publaynet/train/PMC5397437_00004.jpg Parameter ...
/home/daniil/program/dataset/publaynet/train/PMC6087436_00004.jpg 
/home/daniil/program/dataset/publaynet/train/PMC6087436_00004.jpg 
/home/daniil/program/dataset/publaynet/train/PMC4233714_00005.jpg 
/home/daniil/program/dataset/publaynet/train/PMC3644173_00007.jpg 
/home/daniil/program/dataset/publaynet/train/PMC5004597_00008.jpg 
/home/daniil/program/dataset/publaynet/train/PMC5004597_00008.jpg 
/home/daniil/program/dataset/publaynet/train/PMC5029444_00005.jpg 
/home/daniil/program/dataset/publaynet/train/PMC4867865_00005.jpg 
/home/daniil/program/dataset/publaynet/train/PMC6055857_00003.jpg 
/home/daniil/program/dataset/publaynet/train/PMC5773900_00006.jpg syndrome A...
/home/daniil/pro

In [67]:
dataset = {"x": [], "y": []}
for train_i in rez["train"]:
    for x in train_i["x"]:
        dataset["x"].append(x)
    for y in train_i["y"]:
        dataset["y"].append(y)

In [68]:
print("Первый x:\n", dataset["x"][0])
print("Последний x:\n", dataset["x"][-1])
print("Размер x:\t", len(dataset["x"][0]))
print("Первый y:\t", dataset["y"][0])
print("Последний y:\t", dataset["y"][0])
print("Кол-во записей:\t", len(dataset["x"]))

Первый x:
 [0.08622612 0.20952909 0.32334796 0.21094877 1.         1.
 0.37127483 1.         0.82040816 0.51111111 0.32334796 0.20952909
 0.30864255 0.21094877 0.12813077 1.         0.         1.
 0.74285714 0.51111111 1.         0.13968606 0.32334796 0.10547438
 0.         1.         0.12813077 1.         0.68163265 0.04444444
 0.19874139 0.13968606 0.10778265 0.48969605 0.         1.
 0.79489099 0.89442719 0.88571429 0.04444444 0.28921121 0.13968606
 0.08622612 1.         0.         1.         1.         0.96420539
 0.78367347 0.28888889 0.28921121 0.13968606 0.08622612 1.
 0.         1.         1.         0.96420539 0.78367347 0.28888889
 0.28921121 0.13968606 0.08622612 1.         0.         1.
 1.         0.96420539 0.78367347 0.28888889 0.08622612 0.20952909
 0.32334796 0.21094877 1.         1.         0.37127483 1.
 0.82040816 0.51111111 0.28921121 0.13968606 0.08622612 1.
 0.         1.         1.         0.96420539 0.78367347 0.28888889
 0.28921121 0.10476454 0.13112326 0.1757

In [69]:
def shuffle(x_arrays, y_arrays):
    x_vec = np.concatenate(x_arrays)
    y_vec = np.concatenate(y_arrays)
    data = [(x, y) for x, y in zip(x_vec, y_vec)]
    random.shuffle(data)
    x_vec_rnd = np.array([x for x, y in data])
    y_vec_rnd = np.array([y for x, y in data])
    return x_vec_rnd, y_vec_rnd
    

def get_data(dataset, balans=True): #, batch_size=128

    X = np.array(dataset["x"])
    y = np.array(dataset["y"])
    
    keys = np.unique(y)
    Y_keys = dict()
    for i, key in enumerate(keys):
        z = np.zeros(5)
        z[i] = 1
        Y_keys[key] = z
    Y = np.array([Y_keys[yi] for yi in y])

    X_clus_list = []
    Y_clus_list = []

    for i, item in enumerate(Y_keys.keys()):
        X_clus_list.append(X[Y[:,i]==1,:])
        Y_clus_list.append(Y[Y[:,i]==1,:])
        
    if balans:
        balans_len = [len(yc) for yc in Y_clus_list]
        min_len = min(balans_len)
    
    train_x = []
    train_y = []
    test_x = []
    test_y = []
    val_x = []
    val_y = []
    for class_data in zip(X_clus_list, Y_clus_list):
        data = [(x, y) for x, y in zip(class_data[0], class_data[1])]
        random.shuffle(data)
        if balans:
            data = data[:min_len]
        len_data = len(data)
        len_test = round(0.2*len_data)
        len_val = round((len_data-len_test)*0.2)
        len_train = len_data - len_test - len_val
        
        train_x.append([x for x, y in data[:len_train]])
        train_y.append([y for x, y in data[:len_train]])
        
        val_x.append([x for x, y in data[len_train:len_train+len_val]])
        val_y.append([y for x, y in data[len_train:len_train+len_val]])
    
        test_x.append([x for x, y in data[-len_test:]])
        test_y.append([y for x, y in data[-len_test:]])
        
    train_x, train_y = shuffle(train_x, train_y)
    val_x, val_y = shuffle(val_x, val_y)
    test_x, test_y = shuffle(test_x, test_y)
    
    return train_x, train_y, val_x, val_y, test_x, test_y, Y_keys

In [70]:
train_x, train_y, val_x, val_y, test_x, test_y, Y_keys = get_data(dataset, balans=True)

In [71]:
with open("rez.json", "w") as f:
    json.dump({"train_x": train_x.tolist(), "train_y": train_y.tolist(),
               "val_x": val_x.tolist(), "val_y": val_y.tolist(), 
               "test_x": test_x.tolist(), "test_y": test_y.tolist()}, f)

In [19]:
train_x[4]

array([ 0.88235294,  1.        ,  0.6841002 ,  0.36753387,  0.23359112,
        1.        ,  0.        ,  0.        , -0.00985222,  0.98809524,
        0.88235294,  1.        ,  0.6841002 ,  0.36753387,  0.23359112,
        1.        ,  0.        ,  0.        , -0.00985222,  0.98809524,
        0.14705882,  0.05263158,  0.84887469,  0.90068638,  0.10783583,
        1.        ,  0.        ,  1.        ,  0.37438424,  1.        ,
        0.14705882,  0.05263158,  0.84887469,  0.90068638,  0.10783583,
        1.        ,  0.        ,  1.        ,  0.37438424,  1.        ,
        0.11764706,  0.15691705,  0.14705882,  0.96505742,  0.13153341,
        0.92096725,  0.11422282,  0.99067884,  0.39408867,  0.93452381,
        0.14705882,  0.05263158,  0.11764706,  0.28442728,  0.10783583,
        0.99886814,  0.13932398,  1.        ,  0.37438424,  0.88095238,
        0.11764706,  0.01754386,  0.41176471,  0.04740455,  0.61280853,
        0.99654576,  0.23606667,  0.99886814,  0.5270936 ,  0.87

In [2]:
pln_ds = PubLayNetDataset(r"/home/daniil/program/dataset/publaynet/", 
                          r"/home/daniil/program/dataset/tmp_pln")

In [3]:
def fun_from_tmp_and_path_image(tmp_json, img_path):
    doc = Document()
    doc.set_from_path(img_path)
    return {"tmp": tmp_json}


rez = pln_ds.get_json_from_tmps_and_images(fun_from_tmp_and_path_image)

train:


In [4]:
import os, json

In [23]:
path_dataset = r"/home/daniil/program/dataset/mini_ts_publaynet/"
path_dataset_train = os.path.join(path_dataset, "train")

In [8]:
path_train = os.path.join(path_dataset, "train.json")

with open(path_train, "r") as f:
    train_json = json.load(f)

In [11]:

print(train_json["annotations"][0])
print(train_json["images"][0])

{'image_id': 0, 'bbox': [40.42, 444.87, 251.55, 63.86], 'category_id': 1}
{'file_name': 'PMC3866684_00003.jpg', 'height': 811, 'id': 0, 'width': 613}


In [13]:
annotation_image = dict()
for rez in train_json["annotations"]:
    if rez["image_id"] in annotation_image.keys():
        annotation_image[rez["image_id"]].append(rez) 
    else:
        annotation_image[rez["image_id"]] = [rez]

In [24]:
LABEL_PUBLAYNET2IMGDOC = {
    1:1, # text - text
    2:2, # title - header
    3:3, # list - list
    4:4, # table - table
    5:0, # figure -no_struct
}

def fun_additional_info(img_path):
    with open(img_path+".json") as f:
        rez = json.load(f)
    return rez

images = pln_ds._PubLayNetDataset__get_image_train_json(annotation_image.keys())
images_len = len(images)
print(f"кол-во изображений:\t {len(annotation_image)}")
print(f"кол-во сегментов:\t {images_len}")

for i, img in enumerate(images):
    pr = i/images_len*100
    print(f"    {pr:.2f}%    ", end="\r" )
    img_path = os.path.join(path_dataset_train, img["file_name"])
    additional_info = fun_additional_info(img_path)
    blocks = [{"x_top_left":int(aimg["bbox"][0]), 
               "y_top_left":int(aimg["bbox"][1]),
               "width": int(aimg["bbox"][2]), 
               "height":int(aimg["bbox"][3]),
               "label": LABEL_PUBLAYNET2IMGDOC[aimg["category_id"]]
              }for aimg in annotation_image[img["id"]]]
    with open(os.path.join(pln_ds.tmp_path_train_jsons, img["file_name"]+".json"), 'w') as f:
        json.dump({"blocks": blocks, "additional_info": additional_info}, f)

кол-во изображений:	 7514
кол-во сегментов:	 7514
